In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd /content/gdrive/MyDrive/Colab Notebooks/14 - Recommender system

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks/14 - Recommender system


In [ ]:
import numpy as np 
import pandas as pd 
from mlxtend.frequent_patterns import apriori, association_rules
from numpy.random import default_rng

# Apriori

In [ ]:
df = pd.read_csv('./data/Online_Retail.csv', engine='python')
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo', 'Description'], inplace = True)
df = df[~df['Description'].str.contains('wrong|POSTAGE')]
df['InvoiceNo']= df['InvoiceNo'].astype('str')
df = df[df['UnitPrice']!=0]
# Dropping all transactions which were done on credit
df = df[~df['InvoiceNo'].str.contains('C')]
print(df.shape)
df.head()

(528274, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1/12/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,1/12/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1/12/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1/12/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1/12/10 8:26,3.39,17850.0,United Kingdom


In [ ]:
df_France = (df[df['Country'] == "France"] 
        .groupby(['InvoiceNo', 'Description'])['Quantity'] 
        .sum().unstack().reset_index().fillna(0) 
        .set_index('InvoiceNo')
        .clip(upper=1, lower=0))
df_France.columns = ['CMD' + str(x+1000) for x in default_rng().choice(df_France.shape[1], size=df_France.shape[1], replace=False)]
df_France = df_France.loc[(df_France.sum(axis=1)>1).values]
df_France.index.name='SequenceNo'
df_France.head()

,CMD1070,CMD1854,CMD1361,CMD2294,CMD1465,CMD2221,CMD1142,CMD1282,CMD2293,CMD1739,CMD2327,CMD1140,CMD1483,CMD1777,CMD1469,CMD2258,CMD2477,CMD2158,CMD1811,CMD1145,CMD1591,CMD2376,CMD2507,CMD2042,CMD1246,CMD2469,CMD1249,CMD2022,CMD2367,CMD2528,CMD2378,CMD2199,CMD1616,CMD1929,CMD1266,CMD2029,CMD2276,CMD1306,CMD2063,CMD1396,...,CMD1775,CMD2232,CMD1109,CMD1812,CMD1980,CMD2321,CMD1652,CMD1830,CMD2038,CMD2372,CMD2088,CMD1835,CMD2420,CMD2035,CMD1173,CMD1513,CMD1543,CMD1663,CMD2236,CMD1580,CMD1886,CMD1577,CMD1774,CMD1907,CMD2160,CMD2315,CMD2005,CMD2021,CMD2329,CMD1530,CMD1417,CMD1990,CMD2040,CMD1657,CMD1435,CMD1695,CMD1068,CMD1661,CMD2296,CMD2189
SequenceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_France.to_csv('Sequence_Data_format2.csv')

In [ ]:
df = df_France.astype(bool)
df = df.stack().reset_index()
df.columns = ['SequenceNo','AlarmType', 'bool']
df = df[df['bool'] == True].drop(columns=['bool'])
df = df.groupby('SequenceNo')['AlarmType'].apply(list).reset_index()
df.head()
df.to_csv('Sequence_Data.csv')

In [ ]:
# Building the model 
frq_items = apriori(df_France, min_support = 0.02, use_colnames = True) 
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules = rules.loc[(len(x) == 1 for x in rules['consequents'])]
rules['consequents'] = [list(x)[0].replace('CMD', 'Fault ') for x in rules['consequents'].values]
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
419,(CMD1665),Fault 1022,0.024590,0.043716,0.024590,1.000000,22.875000,0.023515,inf
2327,"(CMD1078, CMD1703)",Fault 2253,0.030055,0.043716,0.030055,1.000000,22.875000,0.028741,inf
4654,"(CMD1078, CMD1073, CMD1703)",Fault 2253,0.030055,0.043716,0.030055,1.000000,22.875000,0.028741,inf
2320,"(CMD1073, CMD2253)",Fault 1703,0.035519,0.049180,0.035519,1.000000,20.333333,0.033772,inf
4652,"(CMD1078, CMD1073, CMD2253)",Fault 1703,0.030055,0.049180,0.030055,1.000000,20.333333,0.028577,inf
...,...,...,...,...,...,...,...,...,...
1121,(CMD1528),Fault 1356,0.191257,0.087432,0.021858,0.114286,1.307143,0.005136,1.030319
589,(CMD2231),Fault 1778,0.196721,0.068306,0.021858,0.111111,1.626667,0.008421,1.048156
192,(CMD2231),Fault 1075,0.196721,0.073770,0.021858,0.111111,1.506173,0.007346,1.042008
1194,(CMD2231),Fault 1113,0.196721,0.076503,0.021858,0.111111,1.452381,0.006808,1.038934


In [ ]:
rules.sample(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2178,"(CMD1073, CMD1254)",Fault 1620,0.021858,0.076503,0.021858,1.000000,13.071429,0.020186,inf
5142,"(CMD1572, CMD2470, CMD1415)",Fault 2535,0.038251,0.136612,0.038251,1.000000,7.320000,0.033026,inf
5226,"(CMD1008, CMD1364, CMD1572)",Fault 1415,0.038251,0.060109,0.032787,0.857143,14.259740,0.030488,6.579235
3406,"(CMD1598, CMD1364)",Fault 1415,0.021858,0.060109,0.021858,1.000000,16.636364,0.020544,inf
3809,"(CMD2119, CMD2328)",Fault 1528,0.043716,0.191257,0.021858,0.500000,2.614286,0.013497,1.617486
2837,"(CMD1764, CMD2128)",Fault 1795,0.071038,0.180328,0.021858,0.307692,1.706294,0.009048,1.183971
101,(CMD2484),Fault 2192,0.112022,0.109290,0.038251,0.341463,3.124390,0.026009,1.352560
1475,"(CMD2036, CMD2328)",Fault 1786,0.030055,0.103825,0.024590,0.818182,7.880383,0.021470,4.928962
3958,"(CMD1364, CMD2535)",Fault 1572,0.109290,0.147541,0.106557,0.975000,6.608333,0.090433,34.098361
1338,(CMD1549),Fault 1073,0.169399,0.054645,0.027322,0.161290,2.951613,0.018066,1.127154


# Bayesian Network

In [ ]:
pip install pgmpy

ERROR: Could not find a version that satisfies the requirement pgmpy.estimators (from versions: none)
ERROR: No matching distribution found for pgmpy.estimators


In [ ]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import ParameterEstimator
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.estimators import ConstraintBasedEstimator
from pgmpy.inference import VariableElimination

In [ ]:
data = pd.DataFrame(data={'Problem_C': ["C_Coil", "C_Coil", "C_Coil", "Exp_Motor", 
                                          "Exp_Motor","Exp_Motor", "Exp_Motor", "HS_Switch", 
                                          "HS_Switch", "HS_Switch", "HS_Switch", 
                                          "Exp_Valve", "Exp_Valve", "Exp_Valve",], 
                          'Cause_C': ["trip", "Short circuit", "Unbalance", "Unbalance", 
                                         "Unbalance", "trip", "trip", "trip", 
                                         "Unbalance", "Short circuit", "trip", "Short circuit", 
                                         "Short circuit", "Unbalance"], 
                          'Remedy_C': ["Replace", "Replace", "Replace", "Clean", 
                                          "Clean", "Clean", "Replace","Replace", 
                                          "Clean", "Replace", "Replace", "Clean", 
                                          "Replace", "Clean"]})
print(data)

In [ ]:
data

In [ ]:
model = BayesianModel([('Problem_C', 'Cause_C'), ('Cause_C', 'Remedy_C')])  # ProblemCode -> Cause Code -> Remedy Code
mle = MaximumLikelihoodEstimator(model, data)
print(mle.estimate_cpd('Problem_C'))
print(mle.estimate_cpd('Cause_C'))
print(mle.estimate_cpd('Remedy_C'))
model.fit(data, estimator=MaximumLikelihoodEstimator)

In [ ]:
print(model.nodes())
print(model.edges())
model.get_cpds()

In [ ]:
infer = VariableElimination(model)
p = infer.query(variables=['Remedy_C'], evidence={'Problem_C': 'Exp_Valve'})

infer = VariableElimination(model)
q = infer.query(variables=['Cause_C'], evidence={'Problem_C': 'Exp_Valve'})

print('\n************************************************')
print('Cause & Remedy for Problem Code == Expansion Valve')
print('************************************************\n')
print(q)
print(p)